In [32]:
# libraries needed 
import pandas as pd
import numpy as np
import random
import re
# %matplotlib inline

# libraries we might not need
import csv

### Using the following code
to use the whole document you only need one file specified by filepath for the time being

In [33]:
%%time
# imports a random sample of size s from csv-file as a pandas dataframe
# pandas using python 3.X uses utf-8 encoding

# usage: specify file location, sample size and seed(used by random)
filepath = '../Data_git_ignore/clean_corpus_small.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 12000                    # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

# init dataframe with specified values
df = pd.read_csv(filepath, index_col = [0])

# visual output
#print(df.shape, '<- size of dataframe \n')
df

Wall time: 47 ms


,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,type_id,content_tokenize
1496,338715.0,theinternetpost.net,fake,https://theinternetpost.net/tag/global-war/,we are witnessing a great turning inward in th...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,THE INTERNET POST,NaN,NaN,NaN,Posts about Global War written by ajfloyd and ...,"global conflict, war on terror, United States,...",NaN,NaN,3,"{'-PRON-', 'be', 'witness', 'a', 'great', 'tur..."
8192,30948.0,beforeitsnews.com,fake,http://beforeitsnews.com/spirit/2013/06/more-t...,more than a parade and my new manifesto of rea...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"More than a parade, and my new manifesto",Lisa Beck Living,NaN,NaN,NaN,NaN,NaN,NaN,3,"{'more', 'than', 'a', 'parade', 'and', '-PRON-..."
5154,819308.0,beforeitsnews.com,fake,http://beforeitsnews.com/international/2013/01...,dallas gun buyback program countered with succ...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Dallas Gun Buyback Program Countered With Succ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"{'dallas', 'gun', 'buyback', 'program', 'count..."
2272,339605.0,theinternetpost.net,fake,https://theinternetpost.net/tag/free-trade-area/,by paul j balles many industries that started ...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,THE INTERNET POST,NaN,NaN,NaN,Posts about Free trade area written by ajfloyd,"employment, Free trade agreement, United State...",NaN,NaN,3,"{'by', 'paul', 'j', 'balles', 'many', 'industr..."
2198,24651.0,beforeitsnews.com,fake,http://beforeitsnews.com/business/2015/11/ente...,enterprise vsat market global research and ana...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Enterprise VSAT Market Global Research and Ana...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"{'enterprise', 'vsat', 'market', 'global', 're..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5633,931442.0,christianpost.com,reliable,https://www.christianpost.com/news/democrats-b...,photo reutersjonathan ernst lr republican us s...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Democrats Block Anti-Human Trafficking Bill Ov...,NaN,NaN,NaN,A previously bipartisan U.S. Senate bill meant...,NaN,NaN,NaN,7,"{'photo', 'reutersjonathan', 'ernst', 'lr', 'r..."
5055,930715.0,christianpost.com,reliable,https://www.christianpost.com/news/hillary-cli...,photo reutersbrian snyder democratic president...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Hillary Clinton Says What Obama Won't: ISIS Co...,NaN,NaN,NaN,Democratic presidential front runner and forme...,NaN,NaN,NaN,7,"{'photo', 'reutersbrian', 'snyder', 'democrati..."
5851,157078.0,christianpost.com,reliable,https://www.christianpost.com/news/338-million...,it is becoming known as the NUM million dollar...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,$338 Million Beer Run Makes Lottery History,NaN,NaN,NaN,It is becoming known as the $338 million dolla...,NaN,NaN,NaN,7,"{'-PRON-', 'be', 'become', 'know', 'as', 'the'..."
344,477073.0,nutritionfacts.org,reliable,https://nutritionfacts.org/es/video/ninos-con-...,a continuación una aproximación al contenido d...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Niños con el Estómago Engrapado,Michael Greger M.D. Faclm,NaN,NaN,La cirugía bariátrica para la pérdida de peso ...,"mortalidad, ansiedad, embolia pulmonar, niños,...",NaN,NaN,7,"{'a', 'continuación', 'una', 'aproximación', '..."


In [34]:
df['id'] = pd.to_numeric(df['id'], errors = 'coerce', downcast = 'integer')
df.drop_duplicates(subset = 'id', inplace = True)

In [35]:
df = df.dropna(subset=['id']).set_index('id')
df.shape

(500, 17)

### data-tables: [name]-uniq / relational-tables: [name]_in
creating csv-files for database

In [36]:
# specify where to save all csv-files
path = '../Data_git_ignore/clean_csv/'

# create temporary dataframe and use article id as index 
out_df = pd.DataFrame({'id':df.index})
out_df.set_index('id', inplace=True)

In [37]:
def generateUniqe (category):
    type_array = df[category].unique() # get array of unique types
    type_df = pd.DataFrame({'id': np.arange(type_array.size), 'name':type_array})

    # write file and free memory
    type_df.to_csv(path + category + '.csv', index=False, header=True)
    #del type_array
    #del type_df # tmp delete later

    # create dict with type_name as key - [swap type with type_id]
    type_name_as_key_df = type_df.set_index('name')
    type_dict = type_name_as_key_df['id'].to_dict()

    # replace type with tag id and create new column
    type_id = np.array([type_dict[key] for key in df[category].to_numpy()])
    #df['type_id'] =type_id
    
    return type_id

def generateUniqeArray (category):
    # creates list of list but formaly it is a pd.series of lists
    tags_series_of_lists = df[category].dropna().str.split(', ') # -> ', ' not ','

    if not category in out_df: ### tmp need another method ###
        out_df.insert(0,column = category, value = tags_series_of_lists)

    # flattern tags_series_of_lists to a set(ie. unique values only)
    tags_list = list(set([item for sublist in tags_series_of_lists for item in sublist]))

    # create dataframe
    tags_df = pd.DataFrame({'id': np.arange(len(tags_list)), 'name':tags_list})

    # write file and free memory
    tags_df.to_csv(path + category + '.csv', index=False, header=True)
    del tags_series_of_lists
    del tags_list
    
    # generate the relation
    # get all pairs of article_id and tags in a article (for all articles)
    articles_id_tags_name_pairs_df = out_df[category].dropna().explode().drop_duplicates(keep = 'first')

    # split tags_name and articles_id
    articles_id_array = articles_id_tags_name_pairs_df.index.to_numpy()
    tags_name_array = articles_id_tags_name_pairs_df.to_numpy()

    # create dict with tag_name as key - [swap tags with tags_id]
    tags_name_as_key_df = tags_df.set_index('name')
    tags_dict = tags_name_as_key_df['id'].to_dict()

    # replace tags with tag id
    tags_id = np.array([tags_dict[key] for key in tags_name_array])

    # create dataframe
    tags_in_df = pd.DataFrame(data=articles_id_array, index=tags_id, columns=['article_id'])
    tags_in_df.index.name=category + '_id'

    # write file and free memory
    tags_in_df.to_csv(path + category + '_in.csv', index=True, header=True)

In [38]:
### type_clean - data-table ###
df["type"] = generateUniqe("type")

In [39]:
### tags_clean & tags_in - data-table ###
generateUniqeArray("tags")

In [40]:
### authors_clean & authors_in - data-table ###
generateUniqeArray("authors")

In [41]:
### domain_clean - data-table ###
df["domain"] = generateUniqe("domain")

In [42]:
### meta_keywords_clean & meta_keywords_in - data-table ###
generateUniqeArray("meta_keywords")

In [43]:
# Clean meta keywords representation
regexStrList = r"\["
df['meta_keywords'] = df['meta_keywords'].replace(to_replace=regexStrList, value='{', regex=True)
regexStrList = r"\]"
df['meta_keywords'] = df['meta_keywords'].replace(to_replace=regexStrList, value='}', regex=True)

regexStrList = r"\["
df['content_tokenize'] = df['content_tokenize'].replace(to_replace=regexStrList, value='{', regex=True)
regexStrList = r"\]"
df['content_tokenize'] = df['content_tokenize'].replace(to_replace=regexStrList, value='}', regex=True)

df['content_tokenize']

id
338715    {'-PRON-', 'be', 'witness', 'a', 'great', 'tur...
30948     {'more', 'than', 'a', 'parade', 'and', '-PRON-...
819308    {'dallas', 'gun', 'buyback', 'program', 'count...
339605    {'by', 'paul', 'j', 'balles', 'many', 'industr...
24651     {'enterprise', 'vsat', 'market', 'global', 're...
                                ...                        
931442    {'photo', 'reutersjonathan', 'ernst', 'lr', 'r...
930715    {'photo', 'reutersbrian', 'snyder', 'democrati...
157078    {'-PRON-', 'be', 'become', 'know', 'as', 'the'...
477073    {'a', 'continuación', 'una', 'aproximación', '...
475195    {'información', 'nutricional', 'con', 'el', 'd...
Name: content_tokenize, Length: 500, dtype: object

In [44]:
df.head()

,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,type_id,content_tokenize
id,,,,,,,,,,,,,,,,,
338715,0,0,https://theinternetpost.net/tag/global-war/,we are witnessing a great turning inward in th...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,THE INTERNET POST,NaN,NaN,NaN,Posts about Global War written by ajfloyd and ...,"global conflict, war on terror, United States,...",NaN,NaN,3,"{'-PRON-', 'be', 'witness', 'a', 'great', 'tur..."
30948,1,0,http://beforeitsnews.com/spirit/2013/06/more-t...,more than a parade and my new manifesto of rea...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"More than a parade, and my new manifesto",Lisa Beck Living,NaN,NaN,NaN,NaN,NaN,NaN,3,"{'more', 'than', 'a', 'parade', 'and', '-PRON-..."
819308,1,0,http://beforeitsnews.com/international/2013/01...,dallas gun buyback program countered with succ...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Dallas Gun Buyback Program Countered With Succ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"{'dallas', 'gun', 'buyback', 'program', 'count..."
339605,0,0,https://theinternetpost.net/tag/free-trade-area/,by paul j balles many industries that started ...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,THE INTERNET POST,NaN,NaN,NaN,Posts about Free trade area written by ajfloyd,"employment, Free trade agreement, United State...",NaN,NaN,3,"{'by', 'paul', 'j', 'balles', 'many', 'industr..."
24651,1,0,http://beforeitsnews.com/business/2015/11/ente...,enterprise vsat market global research and ana...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Enterprise VSAT Market Global Research and Ana...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"{'enterprise', 'vsat', 'market', 'global', 're..."


In [45]:
### article clean ###

df[['domain', 'type', "url", "content_tokenize", "content", "title", "meta_keywords", "scraped_at",  "updated_at", "inserted_at"]].to_csv(path + 'article.csv', header=True)

In [46]:
### Setting up wikinews

In [47]:
# imports a random sample of size s from csv-file as a pandas dataframe
# pandas using python 3.X uses utf-8 encoding

# usage: specify file location, sample size and seed(used by random)
filepath = '../Data_git_ignore/wiki_news_backup_clean.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 3679                    # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

# init dataframe with specified values
df = pd.read_csv(filepath, index_col = [0]).sample(n=s, random_state=seed)

# visual output
#print(df.shape, '<- size of dataframe \n')
df.head()

,categories,content,publish_date,scraped_at,source_urls,sources_domain_urls,title,content_tokanize
article_url,,,,,,,,
https://en.wikinews.org/wiki/Namibia_votes_in_presidential_election,"November 28, 2009,Namibia,Published,Elections,...",voters are heading to the polls today in namib...,"November 28, 2009",2020-04-23,http://www1.voanews.com/english/news/africa/27...,"/wiki/VOA_News,/wiki/Associated_Press",Namibia votes in presidential election,"{'voter', 'be', 'head', 'to', 'the', 'poll', '..."
https://en.wikinews.org/wiki/Scientology_defector_arrested_after_attempting_to_leave_organization,"October 9, 2010,Religion,Scientology,Published...",recent reports have revealed that a defector f...,"October 9, 2010",2020-04-22,http://newsfeed.time.com/2010/10/09/scientolog...,"/wiki/TIME,https://en.wikipedia.org/wiki/LA_We...",Scientology defector arrested after attempting...,"{'recent', 'report', 'have', 'reveal', 'that',..."
https://en.wikinews.org/wiki/Nobel_laureate_Doris_Lessing:_9/11_%27wasn%27t_that_terrible%27_compared_to_IRA_attacks,"October 22, 2007,Published,Archived,United Kin...",british writer doris lessing recipient of the ...,"October 22, 2007",2020-04-22,http://www.ctv.ca/servlet/ArticleNews/story/CT...,"https://en.wikipedia.org/wiki/CTV.ca,https://e...",Nobel laureate Doris Lessing: 9/11 'wasn't tha...,"{'british', 'writer', 'doris', 'lessing', 'rec..."
https://en.wikinews.org/wiki/No-fly_zone_demanded_by_Syrian_protesters,"October 29, 2011,Published,Archived,Syria,Midd...",friday saw protests across syria calling for a...,"October 29, 2011",2020-04-22,http://www.bbc.co.uk/news/world-middle-east-15...,"/wiki/BBC_News_Online,/wiki/Agence_France-Pres...",No-fly zone demanded by Syrian protesters,"{'friday', 'see', 'protest', 'across', 'syria'..."
https://en.wikinews.org/wiki/Romanian_oil_company_claims_Iran_has_seized_one_of_its_rigs,"August 22, 2006,Published,AutoArchived,World,P...",an orizont oil platform in the persian gulf ow...,"August 22, 2006",2020-04-22,http://www.bloomberg.com/apps/news?pid=2060108...,"https://en.wikipedia.org/wiki/Bloomberg,/wiki/...",Romanian oil company claims Iran has seized on...,"{'an', 'orizont', 'oil', 'platform', 'in', 'th..."


In [48]:
df['id'] = pd.to_numeric(range(10000000, 10000000+df.shape[0]), errors = 'coerce', downcast = 'integer')
df.drop_duplicates(subset = 'id', inplace = True)

df["category"] = 1

df = df.dropna(subset=['id']).set_index('id')
df.head()

,categories,content,publish_date,scraped_at,source_urls,sources_domain_urls,title,content_tokanize,category
id,,,,,,,,,
10000000,"November 28, 2009,Namibia,Published,Elections,...",voters are heading to the polls today in namib...,"November 28, 2009",2020-04-23,http://www1.voanews.com/english/news/africa/27...,"/wiki/VOA_News,/wiki/Associated_Press",Namibia votes in presidential election,"{'voter', 'be', 'head', 'to', 'the', 'poll', '...",1
10000001,"October 9, 2010,Religion,Scientology,Published...",recent reports have revealed that a defector f...,"October 9, 2010",2020-04-22,http://newsfeed.time.com/2010/10/09/scientolog...,"/wiki/TIME,https://en.wikipedia.org/wiki/LA_We...",Scientology defector arrested after attempting...,"{'recent', 'report', 'have', 'reveal', 'that',...",1
10000002,"October 22, 2007,Published,Archived,United Kin...",british writer doris lessing recipient of the ...,"October 22, 2007",2020-04-22,http://www.ctv.ca/servlet/ArticleNews/story/CT...,"https://en.wikipedia.org/wiki/CTV.ca,https://e...",Nobel laureate Doris Lessing: 9/11 'wasn't tha...,"{'british', 'writer', 'doris', 'lessing', 'rec...",1
10000003,"October 29, 2011,Published,Archived,Syria,Midd...",friday saw protests across syria calling for a...,"October 29, 2011",2020-04-22,http://www.bbc.co.uk/news/world-middle-east-15...,"/wiki/BBC_News_Online,/wiki/Agence_France-Pres...",No-fly zone demanded by Syrian protesters,"{'friday', 'see', 'protest', 'across', 'syria'...",1
10000004,"August 22, 2006,Published,AutoArchived,World,P...",an orizont oil platform in the persian gulf ow...,"August 22, 2006",2020-04-22,http://www.bloomberg.com/apps/news?pid=2060108...,"https://en.wikipedia.org/wiki/Bloomberg,/wiki/...",Romanian oil company claims Iran has seized on...,"{'an', 'orizont', 'oil', 'platform', 'in', 'th...",1


In [49]:
df.index

Int64Index([10000000, 10000001, 10000002, 10000003, 10000004, 10000005,
            10000006, 10000007, 10000008, 10000009,
            ...
            10003669, 10003670, 10003671, 10003672, 10003673, 10003674,
            10003675, 10003676, 10003677, 10003678],
           dtype='int64', name='id', length=3679)

In [50]:
path = '../Data_git_ignore/clean_csv/'

# create temporary dataframe and use article id as index 
out_df = pd.DataFrame({'id':df.index})
out_df.set_index('id', inplace=True)

In [51]:
def generateUniqe (category):
    type_array = df[category].unique() # get array of unique types
    type_df = pd.DataFrame({'id': np.arange(type_array.size), 'name':type_array})

    # write file and free memory
    type_df.to_csv(path + category + '.csv', index=False, header=True)
    #del type_array
    #del type_df # tmp delete later

    # create dict with type_name as key - [swap type with type_id]
    type_name_as_key_df = type_df.set_index('name')
    type_dict = type_name_as_key_df['id'].to_dict()

    # replace type with tag id and create new column
    type_id = np.array([type_dict[key] for key in df[category].to_numpy()])
    #df['type_id'] =type_id
    
    return type_id

def generateUniqeArray (category):
    # creates list of list but formaly it is a pd.series of lists
    tags_series_of_lists = df[category].dropna().str.split(', ') # -> ', ' not ','

    if not category in out_df: ### tmp need another method ###
        out_df.insert(0,column = category, value = tags_series_of_lists)

    # flattern tags_series_of_lists to a set(ie. unique values only)
    tags_list = list(set([item for sublist in tags_series_of_lists for item in sublist]))

    # create dataframe
    tags_df = pd.DataFrame({'id': np.arange(len(tags_list)), 'name':tags_list})

    # write file and free memory
    tags_df.to_csv(path + category + '.csv', index=False, header=True)
    del tags_series_of_lists
    del tags_list
    
    # generate the relation
    # get all pairs of article_id and tags in a article (for all articles)
    articles_id_tags_name_pairs_df = out_df[category].dropna().explode().drop_duplicates(keep = 'first')

    # split tags_name and articles_id
    articles_id_array = articles_id_tags_name_pairs_df.index.to_numpy()
    tags_name_array = articles_id_tags_name_pairs_df.to_numpy()
    
    # create dict with tag_name as key - [swap tags with tags_id]
    tags_name_as_key_df = tags_df.set_index('name')
    tags_dict = tags_name_as_key_df['id'].to_dict()

    # replace tags with tag id
    tags_id = np.array([tags_dict[key] for key in tags_name_array])
    
    # create dataframe
    tags_in_df = pd.DataFrame(data=articles_id_array, index=tags_id, columns=['id'])
    tags_in_df.index.name=category + '_id'

    # write file and free memory
    tags_in_df.to_csv(path + category + '_in.csv', index=True, header=True)

In [52]:
### categoires_clean & categories_in ###
generateUniqeArray("categories")

In [53]:
### sources_domain_urls_clean & sources_domain_urls_in
generateUniqeArray("sources_domain_urls")

In [54]:
# Clean meta keywords representation
regexStrList = r"\["
df['content'] = df['content'].replace(to_replace=regexStrList, value='{', regex=True)
regexStrList = r"\]"
df['content'] = df['content'].replace(to_replace=regexStrList, value='}', regex=True)

In [55]:
### article clean ###
df[['content', 'category', "title", "scraped_at", "publish_date"]].to_csv(path + 'wiki_article.csv', header=True)

In [56]:
df[['content', 'category', "title", "scraped_at", "publish_date"]]

,content,category,title,scraped_at,publish_date
id,,,,,
10000000,voters are heading to the polls today in namib...,1,Namibia votes in presidential election,2020-04-23,"November 28, 2009"
10000001,recent reports have revealed that a defector f...,1,Scientology defector arrested after attempting...,2020-04-22,"October 9, 2010"
10000002,british writer doris lessing recipient of the ...,1,Nobel laureate Doris Lessing: 9/11 'wasn't tha...,2020-04-22,"October 22, 2007"
10000003,friday saw protests across syria calling for a...,1,No-fly zone demanded by Syrian protesters,2020-04-22,"October 29, 2011"
10000004,an orizont oil platform in the persian gulf ow...,1,Romanian oil company claims Iran has seized on...,2020-04-22,"August 22, 2006"
...,...,...,...,...,...
10003674,freelance writer jill carroll currently workin...,1,Reporter abducted in Baghdad Saturday morning,2020-04-22,"January 10, 2006"
10003675,despite heavy snowing tens of thousands of wor...,1,Tens of thousands of workers demonstrate in Lj...,2020-04-22,"November 26, 2005"
10003676,thousands of greeks gathered last night in syn...,1,Silent protest in Athens over recent fires,2020-04-22,"August 30, 2007"
